In [1]:
# this script gives all the small entities

In [12]:
import json
import glob
import os
import re
import pandas as pd
import itertools

In [13]:
def deleteLarge(e):
    # adding index number to a set would be better because set only contains unique values
    delset = set()
    for i,j in itertools.combinations(e,2):
        #print("i is {}".format(i))
        #print("j is {}".format(j))
        x = range(i[0],i[1])
        y = range(j[0],j[1])
        k = list(set(x).intersection(y))
        #print(k)
        if len(k) != 0:
            if len(x) >= len(y):
                delset.add(e.index(j))
            else:
                delset.add(e.index(i))
            #print(i)
            #startindex = k[0]
            #print("-----")
            #if i[0] == startindex:
                #print("Deleting j {}".format(j))
            #    delset.add(e.index(i))
                #delistE.append(e.index(j))
            #else:
                #print("Deleting i {}".format(i))
            #    delset.add(e.index(j))
                #delistE.append(e.index(i))
        #print("------------------------")
    delistE = list(delset)   
    
    for ele in sorted(delistE, reverse = True):
        del e[ele]
    return e

In [14]:
# initializing a dictionary to put all the content
annotations = {}
TRAIN_DATA = []
notworked = []

In [15]:
def extractAnnotation(foldername,namepattern,Name_Entity,entword):
    """
    Argumants example
    foldername = "D:\FYP\Annotation JSON\Inception\JSON fils Suwani"
    namepattern = 'S\_\d\d?\d?\.json'
    Name_Entity: The name of the dictionary where all the entities are saved in the json files 
    """
    file_list = glob.glob(os.path.join(os.getcwd(), foldername, "*.json"))
    #for i in range(500):
    for i in range(len(file_list)):
        print(i)
        print(file_list[i])
        name = re.findall(namepattern, file_list[i])
        print(name[0])
        with open(file_list[i],encoding='utf-8') as data_file:
            data = json.load(data_file)
        # Getting the entire text
        fulltext = data['_referenced_fss']['1']['sofaString']
        # Replacing 
        sentences_all =fulltext.replace("â€¢","\n")
        ## Replacing \r\n with two spaces and \n with one space
        sentences_all = sentences_all.replace('\r\n', '  ').replace('\n',' ')
        # Getting the sentence boundaries
        Sentence = data['_views']['_InitialView']['Sentence']
        # Set first sentence starting position 0
        Sentence[0]['begin'] = 0
        sentences_list = []
        # Making sentence list
        for i in range(len(Sentence)):
            sent = sentences_all[Sentence[i]['begin']:Sentence[i]['end']]
            sentences_list.append(sent)
        try:
            # Extract entity start/ end positions and names
            ent_loc = data['_views']['_InitialView'][Name_Entity]

            # if 'begin' in not present create new key called begin and assign 0
            for i in range(len(ent_loc)):
                #print(ent_loc[i])
                if 'begin' not in ent_loc[i]:
                    ent_loc[i]['begin'] = 0
                   # print(ent_loc[i])
             # if 'Entities' or 'Value' in not present create new key called Entities and assign "Not annotated"
            #entword = 'Entities'
            for i in range(len(ent_loc)):
            #print(ent_loc[i])
                if entword not in ent_loc[i]:
                    ent_loc[i][entword] = 'Not annotated'
               # print(ent_loc[i])

            #TRAIN_DATA = []
            ent_list = []
            for sl in range(len(Sentence)):
                #print(".....................")
                #print(Sentence[sl])
                ent_list_sen = []
                #print(".....................")
                for el in range(len(ent_loc)):
                    if 'begin' not in ent_loc[el]:
                        ent_loc[el]['begin'] = 0
                    #print(ent_loc[el])
                    if(ent_loc[el]['begin'] >= Sentence[sl]['begin'] and ent_loc[el]['end'] <= Sentence[sl]['end']):
                        ent_list_sen.append([(ent_loc[el]['begin']-Sentence[sl]['begin']),(ent_loc[el]['end']-Sentence[sl]['begin']),ent_loc[el][entword]])

                ent_list.append(ent_list_sen)
                ## Create blank dictionary
                ent_dic = {}
                ## To delete the large entity and keep the smaller one
                deleteLarge(ent_list[-1])

                ## Fill value to the dictionary
                ## Only append where the is an entitiy 
                if len(ent_list[-1]) != 0:
                    ent_dic['entities'] = ent_list[-1]
                    ## Prepare final training data
                    TRAIN_DATA.append([sentences_list[sl],ent_dic])
            #name = re.findall(namepattern, file_list[i])
            #print(name)
            annotations[name[0]] = TRAIN_DATA
        except KeyError:
            print("Didnt work for {}".format(name[0]))
            notworked.append(name[0])

In [16]:
extractlist = [extractAnnotation("D:\FYP\Annotation JSON\Inception\JSON fils Suwani",'S\_\d\d?\d?\.json','Name_Entity','Entities'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\JASON files Diluni",'D\_\d\d?\d?\.json','NamedEntity','value'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\Jason Himali",'admin\(\d\d?\d?\)\.json','NamedEntity','value'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\Inception_Harini",'HD\_\d\d?\d?\.json','NER','Entities')]

for i in extractlist:
    i


0
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_1.json
S_1.json
1
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_10.json
S_10.json
2
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_100.json
S_100.json
3
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_101.json
S_101.json
4
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_102.json
S_102.json
5
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_103.json
S_103.json
6
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_104.json
S_104.json
7
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_105.json
S_105.json
8
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_106.json
S_106.json
9
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_107.json
S_107.json
10
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_108.json
S_108.json
11
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_109.json
S_109.json
12
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_11.json
S_11.json
13
D:\FYP\Annotation JSON\Inception\JSON fi

232
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_308.json
S_308.json
233
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_309.json
S_309.json
234
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_31.json
S_31.json
235
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_310.json
S_310.json
236
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_311.json
S_311.json
237
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_312.json
S_312.json
238
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_313.json
S_313.json
239
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_314.json
S_314.json
240
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_315.json
S_315.json
241
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_316.json
S_316.json
242
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_317.json
S_317.json
243
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_318.json
S_318.json
244
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_319.json
S_319.json
245
D:\FYP\Ann

361
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_424.json
S_424.json
362
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_425.json
S_425.json
363
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_426.json
S_426.json
364
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_427.json
S_427.json
365
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_428.json
S_428.json
366
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_429.json
S_429.json
367
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_43.json
S_43.json
368
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_430.json
S_430.json
369
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_431.json
S_431.json
370
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_432.json
S_432.json
371
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_433.json
S_433.json
372
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_434.json
S_434.json
373
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_435.json
S_435.json
374
D:\FYP\Ann

39
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_134.json
D_134.json
40
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_135.json
D_135.json
41
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_136.json
D_136.json
42
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_137.json
D_137.json
43
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_138.json
D_138.json
44
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_139.json
D_139.json
45
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_14.json
D_14.json
46
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_140.json
D_140.json
47
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_141.json
D_141.json
48
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_142.json
D_142.json
49
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_143.json
D_143.json
50
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_144.json
D_144.json
51
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_145.json
D_145.json
5

D_267.json
187
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_268.json
D_268.json
188
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_269.json
D_269.json
189
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_27.json
D_27.json
190
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_270.json
D_270.json
191
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_271.json
D_271.json
192
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_272.json
D_272.json
193
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_273.json
D_273.json
194
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_274.json
D_274.json
195
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_275.json
D_275.json
196
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_276.json
D_276.json
197
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_277.json
D_277.json
198
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_278.json
D_278.json
199
D:\FYP\Annotation JSON\Inception\JASON files Diluni

...

224
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(302).json
admin(302).json
225
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(303).json
admin(303).json
226
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(304).json
admin(304).json
227
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(305).json
admin(305).json
228
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(306).json
admin(306).json
229
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(307).json
admin(307).json
230
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(308).json
admin(308).json
231
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(309).json
admin(309).json
232
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(31).json
admin(31).json
233
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(310).json
admin(310).json
234
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(311).json
admin(311).json
235
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(312).json
admin(312).json
236
D:\FYP\Annotat

344
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(412).json
admin(412).json
345
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(413).json
admin(413).json
346
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(414).json
admin(414).json
347
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(415).json
admin(415).json
348
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(416).json
admin(416).json
349
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(417).json
admin(417).json
350
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(418).json
admin(418).json
351
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(419).json
admin(419).json
352
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(42).json
admin(42).json
353
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(420).json
admin(420).json
354
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(421).json
admin(421).json
355
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(422).json
admin(422).json
356
D:\FYP\Annotat

D:\FYP\Annotation JSON\Inception\Jason Himali\admin(83).json
admin(83).json
482
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(84).json
admin(84).json
483
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(85).json
admin(85).json
484
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(86).json
admin(86).json
485
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(87).json
admin(87).json
486
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(88).json
admin(88).json
487
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(89).json
admin(89).json
488
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(9).json
admin(9).json
489
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(90).json
admin(90).json
490
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(91).json
admin(91).json
491
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(92).json
admin(92).json
492
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(93).json
admin(93).json
493
D:\FYP\Annotation JSON\Inception\Jason Him

180
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_261.json
HD_261.json
181
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_262.json
HD_262.json
182
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_263.json
HD_263.json
183
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_264.json
HD_264.json
184
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_265.json
HD_265.json
185
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_266.json
HD_266.json
186
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_267.json
HD_267.json
187
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_268.json
HD_268.json
188
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_269.json
HD_269.json
189
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_27.json
HD_27.json
190
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_270.json
HD_270.json
191
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_271.json
HD_271.json
192
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_272.json


HD_458.json
399
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_459.json
HD_459.json
400
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_46.json
HD_46.json
401
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_460.json
HD_460.json
402
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_461.json
HD_461.json
403
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_462.json
HD_462.json
404
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_463.json
HD_463.json
405
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_464.json
HD_464.json
406
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_465.json
HD_465.json
407
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_466.json
HD_466.json
408
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_467.json
HD_467.json
409
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_468.json
HD_468.json
410
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_469.json
HD_469.json
411
D:\FYP\Annotation JSON\Inception\Inception_Harini\

In [17]:
notworked

[]

In [18]:
len(annotations)

1998

In [19]:
TRAIN_DATA

[['Buddhist and Pali University of Sri Lanka  Invitation for Applications/Nominations  POST OF VENERABLE VICE-CHANCELLOR  Applications are invited from eminently qualified Bhikkus who are well versed in  Discipline and Doctrine in Buddhist Studies and Pali to take over the leadership  of the Buddhist and Pali University of Sri Lanka as its Venerable Vice-Chancellor  according to the provisions of the Buddhist and Pali University Act No. 74 of 1981 as  amended by the Buddhist and Pali University of Sri Lanka (Amendment) Act No. 37  of 1995.',
  {'entities': [[0, 28, 'company']]}],
 ['The Head Office of the university is located at No 37, Moragahahena Road, Pitipana,  Homagama.',
  {'entities': [[48, 93, 'address']]}],
 ['Venerable Vice-Chancellor is a full time officer of the university.',
  {'entities': [[10, 25, 'job_title']]}],
 ['The applicant should not be over 62 years of age as at 31.10.2020.',
  {'entities': [[33, 41, 'no_years']]}],
 ['The Venerable Vice-Chancellor will be resp

In [20]:
import pickle

with open('TrainingDataLargeEntity', 'wb') as fp:
    pickle.dump(TRAIN_DATA, fp)

In [54]:
with open ('TrainingData', 'rb') as fp:
    itemlist = pickle.load(fp)

In [24]:
len(itemlist)

10876

In [30]:
#suwanidf = pd.DataFrame(TRAIN_DATA, columns = ['Text','Annotation'])

In [20]:
#suwanidf = pd.DataFrame(annotations.items(), columns = ['name','annotations'] )

In [31]:
#suwanidf.to_csv(r"D:\FYP\Annotation JSON\Inception\SuwaniALLfromtraininglist.csv")